In [ ]:
from sklearn import preprocessing
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
#Loading the required 10sec_to_30sec files
X_train_30sec_F = pd.read_csv('/home/satishk/GAN_lre/gan_csv/10sec_to_30sec_data/X_train_30sec_F.csv')

y_30sec_labels_train = pd.read_csv('/home/satishk/GAN_lre/gan_csv/10sec_to_30sec_data/y_30sec_labels_train.csv')

train_X_gen_30 = pd.read_csv('/home/satishk/GAN_lre/gan_csv/10sec_to_30sec_data/train_X_gen.csv')

X_val_gen_30 = pd.read_csv('/home/satishk/GAN_lre/gan_csv/10sec_to_30sec_data/X_val_gen.csv')

y_val_labels_30 = pd.read_csv('/home/satishk/GAN_lre/gan_csv/10sec_to_30sec_data/y_val_labels.csv')



#Loading the required 3sec_to_10sec files
train_X_gen_10 = pd.read_csv('/home/satishk/GAN_lre/gan_csv/3sec_to_10sec_data/train_X_gen.csv')

y_10sec_labels_train = pd.read_csv('/home/satishk/GAN_lre/gan_csv/3sec_to_10sec_data/y_10sec_labels_train.csv')

X_val_gen_10 = pd.read_csv('/home/satishk/GAN_lre/gan_csv/3sec_to_10sec_data/X_val_gen.csv')

y_val_labels_10 = pd.read_csv('/home/satishk/GAN_lre/gan_csv/3sec_to_10sec_data/y_val_labels.csv')

train_X_gen_F = pd.concat([train_X_gen_30, train_X_gen_10, X_train_30sec_F], axis=0)



y_labels_train_F = pd.concat([y_30sec_labels_train, y_10sec_labels_train, y_30sec_labels_train], axis=0)


val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_07Feb_30sec.csv')


X_val_30_orig = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val_30_orig = val_lre["language_code"]


le = preprocessing.LabelEncoder()
le.fit(y_val_30_orig)

y_val_labels_30_orig = le.transform(y_val_30_orig)


y_val_labels_30_orig2 =y_val_labels_30_orig.reshape(933,1)



y_val_labels_30 = pd.DataFrame(y_val_labels_30.values)
y_val_labels_10 = pd.DataFrame(y_val_labels_10.values)
y_val_labels_30_orig2 = pd.DataFrame(y_val_labels_30_orig2)

y_labels_val_F = pd.concat([y_val_labels_30, y_val_labels_10, y_val_labels_30_orig2], axis=0)



X_val_F = pd.concat([X_val_gen_30, X_val_gen_10, X_val_30_orig], axis=0)



train_X_gen_F = train_X_gen_F.values
y_labels_train_F = y_labels_train_F.values
X_val_F = X_val_F.values
y_labels_val_F = y_labels_val_F.values



y_labels_train_F = y_labels_train_F.reshape(470946,)
y_labels_val_F = y_labels_val_F.reshape(2787,)


#To train the SVM on small dataset uncomment the below 2 lines

#train_X_gen_F = train_X_gen_F[0:150000]
#y_labels_train_F = y_labels_train_F[0:150000]


#Training a SVM classifier
classif = SVC(C=1, kernel='poly', degree=2, gamma='auto', coef0=1, shrinking=True, random_state=0,
              probability=True, tol=1e-3, cache_size=1e4, class_weight='balanced')
classif.fit(train_X_gen_F, y_labels_train_F)

acc = accuracy_score(y_labels_val_F, classif.predict(X_val_F))

print('accuracy',acc)


#Saving the trained SVM
from sklearn.externals import joblib

filename = '/home/satishk/GAN_lre/gan_csv/svm_poly_prob_true.joblib.pkl'

_ = joblib.dump(classif, filename, compress=9)
print('SVM Saved...')

In [1]:
#Loading the saved classifier
#Path to Saved SVM
filename = '/home/satishk/GAN_lre/gan_csv/svm_poly_prob_true.joblib.pkl'
clf2 = joblib.load(filename)

#Load the eval data or use ur 
X_eval = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/eval_feat_BNF_h5_25451_07Nov.csv')

X_eval.rename(columns={'uttid':'segmentid'}, inplace=True)

X_eval_f = X_eval.drop(["segmentid"],axis=1)

y_eval_segmentid = X_eval["segmentid"]

X_eval_f = X_eval_f.values




#Generating the Score file for Cavg:Enter the eval data in mumpy format in palce of 'X_eval"

posteriors = clf2.predict_proba(X_eval)

log_posteriors = np.log(posteriors)

log_likelihood = log_posteriors

df_post = pd.DataFrame(log_likelihood)

df_post.columns = ['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg', 'por-brz', 'qsl-pol','qsl-rus', 'spa-car', 'spa-eur', 'spa-lac', 'zho-cmn','zho-nan']

#df_y_eval_segmentid=  pd.DataFrame(y_eval_segmentid)
df_y_val_segmentid=  pd.DataFrame(y_val_segmentid)

#yy = pd.concat([df_y_eval_segmentid,df_post],axis=1)
yy = pd.concat([df_y_val_segmentid,df_post],axis=1)


yy.to_csv('/home/satishk/ivectors_csv_new/test_val1_20Feb.csv',index=False)

test_dev_scr = pd.read_csv('/home/satishk/ivectors_csv_new/test_val1_20Feb.csv')


#test_dev_scr = test_dev_scr.drop("segmentid.1",axis=1)

test_dev_scr.to_csv('/home/satishk/odyssey_scores/test_val1_20Feb.scr', header=True, index=False, sep='\t')

print ("Scoring file saved to '/home/satishk/odyssey_scores' folder:")

